In [ ]:
import pandas as pd, numpy as np, matplotlib.dates as dates, seaborn as sns, paramiko, os, re, tables, glob, sys, shutil
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from ipywidgets import interact, IntSlider
if "./src" not in sys.path:
    sys.path.insert(0, "./src")
from src import bcm_utils

# 0) Define parameters

In [ ]:
#variables
data_path = "/Users/pkicsiny/sshfs/json_pickles"
year = 2018
months = [str(m).zfill(2) for m in list(range(4,11))]
rs = 10
colnames = bcm_utils.get_column_names([24, 48],[rs])

#dirs
excel_brildata_path = "../excel_brildata/{}".format(year)
excel_json_path = "../excel_json/{}".format(year)
brilcalc_dir = "../brilcalc_data/{}.csv".format(year)

#files in dirs
excel_json_files = os.listdir(excel_json_path)
excel_json_charge_sums = [excel_json_files[idx]for idx, i in enumerate(["sums" in et for et in excel_json_files]) if i]
excel_json_charge_sum_ratios = [excel_json_files[idx]for idx, i in enumerate(["sum_ratio" in et for et in excel_json_files]) if i]
excel_json_sim_data_ratios = [excel_json_files[idx]for idx, i in enumerate(["sim_data" in et for et in excel_json_files]) if i]

# 1) Read monthly json pickles

In [ ]:
pickle_files = os.listdir(data_path)
pickle_file_dict = {m: [pf for pf in data_path if "_{}_".format(m) in pf] for m in months}
result_df = {m: pd.DataFrame() for m in months}

In [ ]:
for m in ["08"]:#months:
    print("Month {}".format(m))
    result_df[m] = pd.read_pickle(data_path+"/pickle_{}".format(m))/bcm_utils.γ[9]/bcm_utils.β
    result_df[m].index = [pd.Timestamp(ts).tz_localize(None) for ts in result_df[m].index]
    
    break

In [ ]:
for m in ["08"]: #months:
    print(m)
    bcm_utils.plot_blm_data(result_df[m],
              hours=dates.HourLocator(interval=48),
              h_fmt=dates.DateFormatter('%m/%d'), save=True)
    
    break

# 2) Read excel brildata

In [ ]:
excel_df = {m: pd.DataFrame() for m in months}
for m in months:
    excel_df[m] = pd.read_excel(os.path.join(excel_brildata_path, "charge_sums_2018_{}.xlsx".format(m)))

In [ ]:
m = "10"
print("From brildata:\n",excel_df[m][["CH24RS12", "CH48RS12"]].sum())
print("From json:\n",result_df[m].sum())

# 3) Read brilcalc reference data

In [ ]:
#read excel file and convert lumi data from [μb] to [b]
brilcalc_table = pd.read_csv(brilcalc_dir, header=1)[:-3]
brilcalc_table["delivered_[b]"] = pd.to_numeric(brilcalc_table["delivered(/ub)"], errors="coerce")*1e6
brilcalc_table[['run','fill']] = brilcalc_table['#run:fill'].str.split(':',expand=True)
brilcalc_table = brilcalc_table.sort_index()

In [ ]:
#sum up delivered lumi per fill, used to filter hd5 files for fills that are in this table
brilcalc_lumi_table = pd.DataFrame({"fill": pd.to_numeric(brilcalc_table["fill"]).unique().tolist(),
                                    "start_time": [pd.Timestamp(gr.iloc[0]) for gr in pd.DataFrame(brilcalc_table["time"].groupby(brilcalc_table["fill"]))[1]],
                                    "cms_delivered": [brilcalc_table[brilcalc_table["fill"] == fill]["delivered_[b]"].sum()\
                                                      for fill in brilcalc_table["fill"].unique()]})

In [ ]:
bcm_utils.freeze_header(brilcalc_lumi_table)

# 4) Group json data by fills based on brilcalc reference times

In [ ]:
#data can cross month boundaries, always regard fill start date
result_annual_df = pd.concat([result_df[m] for m in months])
charge_sum_df = {m: pd.DataFrame() for m in months}
charge_sum_error_df = {m: pd.DataFrame() for m in months}
bl_noise_list = []

excel_json_baseline = "../baseline_noise/bl_2018.xlsx"

In [ ]:
"""
baseline noise is once per month by using a sample fill which has baseline well visible in the time series
sample fills:
04: 6545
05: 6699
06: 6797
07: 6874
08: 7008
09: 7151
10: 7392
"""

brilcalc_lumi_pp = brilcalc_lumi_table[brilcalc_lumi_table["start_time"] < "2018-11-01 00:00:00"]
tot_len = len(brilcalc_lumi_pp)
save_plot = False

#loop over fill start dates in brilcalc df
for idx in range(tot_len):
    fill = brilcalc_lumi_pp["fill"][idx]
    print("[{}/{}] Current fill: {}".format(idx+1, tot_len, fill))

#all except the last fill
    try:
        fill_data = result_annual_df.loc[brilcalc_lumi_pp["start_time"][idx]:brilcalc_lumi_pp["start_time"][idx+1]]
        
#last fill
    except:
        fill_data = result_annual_df.loc[brilcalc_lumi_pp["start_time"][idx]:]

#save fill time series for baseline noise calculation
    if fill in [6545, 6699, 6797, 6874, 7008, 7151, 7392]:
        bl_noise_list.append(fill_data)
        
#sum up charge and append to df of right month
    month = str(fill_data.index[0])[5:7]
    print("Summing up collected charge.")
    fill_charge_sum = pd.DataFrame(fill_data.sum()).T
    fill_charge_sum.index = [fill]
    charge_sum_df[month] = charge_sum_df[month].append(fill_charge_sum)
    
#calculate stat uncertainty of charge sum
    baseline_excel = pd.read_excel(os.path.join(excel_json_baseline), index_col="month")
    baseline_values = baseline_excel.loc[int(month)][["ch24", "ch48"]]
    stat_unc = pd.Series(baseline_values).rename({"ch24": "CH24RS9", "ch48": "CH48RS9"})*np.sqrt(len(fill_data))
    #relative_unc = stat_unc/fill_data.sum()*100
    fill_charge_sum_error = pd.DataFrame(pd.Series(stat_unc)).T
    fill_charge_sum_error.index = [fill]
    charge_sum_error_df[month] = charge_sum_error_df[month].append(fill_charge_sum_error)


#make plot of fill data
    if save_plot:
        print("Saving plot of fill.")
        xticks = round(round(len(fill_data)/3600)/10)
        plot_blm_data(fill_data,
                      save=True,
                      fill=fill,
                      hours=dates.HourLocator(interval=max(1, xticks)),
                      h_fmt=dates.DateFormatter('%m/%d-%Hh'))
        plt.close()

In [ ]:
#baseline noise in the two channels
fill_noise = bl_noise_list[idx][-6000:-4000].std()*len(bl_noise_list[idx])
fill_sum = bl_noise_list[idx].sum()
relative_noise = fill_noise/fill_sum*100
print(relative_noise, "values in %")

In [ ]:
bcm_utils.freeze_header(charge_sum_error_df["04"])

In [ ]:
#save to excel
if not os.path.isdir(excel_json_path):
        os.makedirs(excel_json_path, exist_ok=True)
        
[charge_sum_df[m].rename(columns={"CH24RS9":"CH24RS10", "CH48RS9":"CH48RS10"}).to_excel(
    excel_json_path+"/charge_sums_{}_{}.xlsx".format(year, m)) for m in months]

In [ ]:
#save stat uncertainties to excel
if not os.path.isdir(excel_json_path):
        os.makedirs(excel_json_path, exist_ok=True)
        
[charge_sum_error_df[m].rename(columns={"CH24RS9":"CH24RS10", "CH48RS9":"CH48RS10"}).to_excel(
    excel_json_path+"/charge_sums_stat_uncertainties_{}_{}.xlsx".format(year, m)) for m in months]